**Системи Data Science. Лабораторна робота №1.**

Виконав: Донченко Богдан 

Група: КМ-81

**Область дослідження:**
Визначимося з предметною областю для дослідження. Для дослідження були взяті різноманітні аніме(японська анімація) які виходили до 2018 року. Мета цього дослідження - це дізнатися, як кількість глядачів може вплинути на рейтинг аніме та дізнатися, як кількість епізодів в аніме впливає на рейтинг.

Підключаємо бібліотеки для полегшення роботи з даними.

In [ ]:
import numpy as np #Бібліотека для математичних розрахунків
import pandas as pd #Бібліотека для роботи з даними
import matplotlib.pyplot as plt #Бібліотека для побудови графіків
import seaborn as sns #Бібліотека для побудови графіків
from sklearn.preprocessing import LabelEncoder #Клас взятий з бібліотеки, для зміни типу даних

Для завдання буде використан ДатаСет з даними про різні аніме. Сам ДатаСет взятий з сайту Kaggle. Посилання на ДатаСет: https://www.kaggle.com/CooperUnion/anime-recommendations-database

В цьому блоці ми імортуємо файли з даними в Пайтон та зчитуємо їх за допомогою бібліотеки Pandas

In [ ]:
anime_path = "/kaggle/input/anime-recommendations-database/anime.csv"
rating_path = "/kaggle/input/anime-recommendations-database/rating.csv"

anime_data = pd.read_csv(anime_path)
y_data = pd.read_csv(rating_path)


In [ ]:
anime_data


In [ ]:
y_data

Групуємо дані та приводимо їх в гарний вид.

In [ ]:
y_data_group = y_data.groupby(["anime_id"])["rating"].sum().reset_index(name="rating")
users_data = y_data.groupby(["anime_id"])["user_id"].count().reset_index(name="user_id")
y_data_group['rating'] = y_data_group['rating']/users_data["user_id"]
y_data_group["rating"] = np.rint(y_data_group['rating']).astype('int')
y_data_group

В цій частині коду ми змінюємо тип даних в стовпці "type", та заміняємо значення "Unknown" в стовці "episodes" на 1000. Виконуємо це для полегшення роботи з цими даними.

In [ ]:
le = LabelEncoder()
type_column = list(anime_data["type"].values)
anime_data["type"] = le.fit(type_column).transform(type_column)
anime_data["episodes"] = anime_data["episodes"].replace(["Unknown"], 1000)
anime_data["episodes"] = anime_data["episodes"].astype("int")

Після замін ми можемо побудувати кореляційну матрицю, за допомогою бібліотеки Seaborn

In [ ]:
cor_matrix = anime_data.corr()
plt.subplots()
sns.heatmap(cor_matrix, square = True)

Завдяки кореляційній матриці ми можемо побачити залежність між стовцями. Наприклад: залежність між "rating" та "members" велика, а залежності між "anime_id" та "rating" взагалі нема.

Далі ми об'єднуємо дві таблиці в одну за допомогою бібліотеки Pandas.

In [ ]:
anime_data = pd.merge(anime_data, y_data_group, on = "anime_id", how = 'inner')
anime_data

Після цього ми сортуємо дані по стовпцю "anime_id" в обох таблицях і видаляємо непотрібні стовці.

In [ ]:
anime_data = anime_data.sort_values(['anime_id'], ascending=True)
anime_data = anime_data.drop(columns=["rating_x", "rating_y", "genre", 'name', "anime_id"])
y_data_group = y_data_group.sort_values(["anime_id"], ascending=True)
y_data_group = y_data_group.drop(columns=["anime_id"])
y_data_group = y_data_group[:-6]
anime_data = anime_data.drop(anime_data[anime_data["members"]>800000].index)
anime_data

In [ ]:
y_data_group

In [ ]:
anime_data.shape, y_data_group.shape

Будуємо графіки.

In [ ]:
plt.scatter(anime_data["members"], y_data_group["rating"])
plt.xlabel("Members")
plt.ylabel("Rating")

In [ ]:
plt.scatter(anime_data["episodes"], y_data_group["rating"])
plt.xlabel("Episodes")
plt.ylabel("Rating")
#plt.xticks("off")

In [ ]:
plt.scatter(anime_data["type"], y_data_group["rating"])
plt.xlabel("Type")
plt.ylabel("Rating")

In [ ]:
plt.scatter(anime_data["members"], y_data_group["rating"])
sns.regplot(x = anime_data["members"], y = y_data_group["rating"], data = anime_data, scatter = False)
plt.xlabel("Members")
plt.ylabel("Rating")

In [ ]:
plt.scatter(anime_data["type"], y_data_group["rating"])
sns.regplot(x = anime_data["type"], y = y_data_group["rating"], data = anime_data, scatter = False)
plt.xlabel("Type")
plt.ylabel("Rating")

In [ ]:
plt.scatter(anime_data["episodes"], y_data_group["rating"])
sns.regplot(x = anime_data["episodes"], y = y_data_group["rating"], data = anime_data, scatter = False)
plt.xlabel("Episodes")
plt.ylabel("Rating")

Висновок: після виконання всіх операції та побудови графіків, ми можемо побачити, що чим більше глядачів дивляться аніме, тим його рейтинги вищі, та навпаки, чим менше глядачів, тим менші рейтинги. Також за графіком ми можемо побачити, що кількість епізодів аніме, майже не впливає на його рейтинг.
